In [1]:
import  pandas as pd
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("titanic")
    .config(
        "spark.jars.packages",
        "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.6.1,"
        "org.apache.hadoop:hadoop-aws:3.3.4"
    )
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.type", "hadoop")
    .config("spark.sql.catalog.demo.warehouse", "s3a://bronze/warehouse/")
    .config("spark.sql.catalog.silver", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.silver.type", "hadoop")
    .config("spark.sql.catalog.silver.warehouse", "s3a://silver/warehouse/")
    .config("spark.sql.catalog.gold", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.gold.type", "hadoop")
    .config("spark.sql.catalog.gold.warehouse", "s3a://gold/warehouse/")        
    .config("spark.hadoop.fs.s3a.access.key", "teste")
    .config("spark.hadoop.fs.s3a.secret.key", "password")
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

%load_ext 00-prettytables

your 131072x1 screen size is bogus. expect trouble
25/10/12 19:33:45 WARN Utils: Your hostname, DESKTOP-4LPUPP7 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/12 19:33:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/andre/miniforge3/envs/iceberg/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/andre/.ivy2/cache
The jars for the packages stored in: /home/andre/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b5f89e7a-593a-467f-96a4-49c36df50099;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 208ms :: artifacts dl 10ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	------------------

✅ Spark SQL Magic carregado! Use %sql ou %%sql para executar consultas Spark SQL.


In [4]:
%%sql
show databases from demo; 

25/10/12 19:34:12 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


namespace
db_teste_iceberg
ml_titanic_raw


In [10]:
%%sql
show tables from  demo.db_teste_iceberg;

namespace,tableName,isTemporary
db_teste_iceberg,teste,False


In [11]:
%%sql
show databases from silver; 

namespace
ml_titanic_silver


In [12]:
%%sql
show tables from  silver.ml_titanic_silver;

namespace,tableName,isTemporary
ml_titanic_silver,titanic,False


In [14]:
%%sql 
select * from silver.ml_titanic_silver.titanic limit 10;

passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,is_child
91,0,3,"Christmann, Mr. Emil",1,29.0,0,0,343276,8.05,-1,s,0
264,0,1,"Harrison, Mr. William",1,40.0,0,0,112059,0.0,B94,s,0
26,1,3,"Asplund, Mrs. Carl Oscar (Selma Augusta Emilia Johansson)",0,38.0,1,5,347077,31.3875,-1,s,0
43,0,3,"Kraeff, Mr. Theodor",1,29.7,0,0,349253,7.8958,-1,c,0
115,0,3,"Attalah, Miss Malake",0,17.0,0,0,2627,14.4583,-1,c,1
99,1,2,"Doling, Mrs. John T (Ada Julia Bone)",0,34.0,0,1,231919,23.0,-1,s,0
127,0,3,"McMahon, Mr. Martin",1,29.7,0,0,370372,7.75,-1,q,0
136,0,2,"Richard, Mr. Emile",1,23.0,0,0,SC/PARIS 2133,15.0458,-1,c,0
191,1,2,"Pinsky, Mrs. (Rosa)",0,32.0,0,0,234604,13.0,-1,s,0
276,1,1,"Andrews, Miss Kornelia Theodosia",0,63.0,1,0,13502,77.9583,D7,s,0
